# AutoGen Lab: Let's Build a Multi-Agent AI Startup


## 🧪1.Lab Introduction

Welcome to the **AutoGen Learning Lab**! In this lab, you'll simulate building a next-generation AI application using the **AutoGen framework** from Microsoft.

Our use case is building an startup team using AI Agents. By the end, you'll not only familiar with AutoGen framework, but also **build a full AI Startup Team with multiple agents** that can work together to solve a complex task.


## 🧠 2. AutoGen Framework

### 2.1 What is AutoGen Framework?

**AutoGen** is an open-source programming framework for building AI agents and facilitating cooperation among multiple agents to solve tasks. AutoGen aims to provide an easy-to-use and flexible framework for accelerating development and research on agentic AI.

### 2.2 Why Use AutoGen?

The AutoGen offers a complete toolkit for designing, implementing, and managing AI agent systems, with a special emphasis on multi-agent collaboration. It abstracts away the complexity of prompt engineering, role-based coordination, and long-context memory management, allowing developers to focus on high-level workflows.

Key Reasons to Use AutoGen:

- Multi-Agent Collaboration: AutoGen allows you to create applications where multiple AI agents, each with specialized roles, can communicate and cooperate to solve complex tasks-much like a human team. This is ideal for scenarios such as customer service bots, data analysis, content generation, and more.

- Simplifies Complex Workflows: It streamlines the orchestration, automation, and optimization of intricate LLM workflows, making it easier to build, manage, and scale applications that would otherwise require significant engineering effort.

- Customizable and Flexible: Developers can easily define agents with specific skills and behaviors, customize conversation patterns, and integrate human feedback when needed. This flexibility enables a wide range of use cases and allows for both autonomous and human-in-the-loop workflows.

- Efficient Code Execution: AutoGen supports automated code execution in secure, containerized environments, enabling agents to write, test, and debug code collaboratively and autonomously-speeding up development and reducing manual intervention.

- Reduces Errors and Increases Consistency: By automating repetitive tasks and providing robust coordination between agents, AutoGen helps reduce human error and ensures consistent results across projects.

- Open Source and Actively Developed: Backed by Microsoft and an active research community, AutoGen is open source, well-documented, and continuously improved, making it accessible and reliable for developers.


## 📚 3. Key Concepts Deep Dive


### 3.1 LLM Config

In AutoGen, llm_config is a dictionary that defines how an agent interacts with a large language model (LLM) such as GPT-4o. It controls the agent’s model behavior, including:

- Which model to use (e.g., OpenAI GPT-4, Azure OpenAI Model)
- Endpoint settings and API configuration
- Model generation style (e.g., creative vs deterministic)
- Tool usage (e.g., code execution tools)

It acts as a bridge between the agent and the LLM backend.


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Load the configuration for Azure OpenAI from .env file.
# This includes the API key, endpoint, and other Azure-specific settings
config_list = [
    {
        "model": os.getenv("AZURE_OPENAI_MODEL"),
        "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
        "base_url": os.getenv("AZURE_OPENAI_ENDPOINT"),
        "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
        "api_type": "azure",
    }
]

In [ ]:
llm_config = {
    "config_list": config_list,  # Use the Azure OpenAI configuration
    "seed": 42,  # Set a seed for caching and reproducibility
}

### 3.2 Base Agent Creation

Agents are the fundamental units of intelligence in AutoGen. Each agent is:

- Configurable: Can be given different roles, prompts, memory, and toolsets.
- Conversational: Can send and receive structured messages.
- Autonomous: Makes decisions based on incoming messages and internal logic.


In [ ]:
from autogen import UserProxyAgent, AssistantAgent

In [ ]:
# Create a User Proxy agent that represents the user
user = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",  # Allow human input before termination
    code_execution_config={
        "use_docker": False,  # Don't use Docker for code execution
    },
    system_message="You are a user to provide requirements for the assistant, when you get the assistant's response, you can provide your own opinion and ask the assistant to improve the idea.",
)

In [ ]:
# Create a assistant agent that will help with tasks
assistant = AssistantAgent(name="Assistant", llm_config=llm_config)

### 3.3 Orchestration

Orchestration is the art of coordinating multiple agents to solve a larger problem.

This is achieved via:

🧩 GroupChat

- Stores all agent interactions and messages
- Ensures shared memory and message visibility

🧠 GroupChatManager

- Selects which agent responds next
- Controls round limits
- Optional methods for speaker logic, stop conditions, etc.


In [ ]:
from autogen import GroupChat, GroupChatManager

group = GroupChat(
    agents=[user, assistant],
    messages=[],
    max_round=5,
    speaker_selection_method="round_robin",
)
manager = GroupChatManager(groupchat=group, llm_config=llm_config)

### 3.4 Entry Point

The entry point for agentic orchestration, where the UserProxyAgent initiates a multi-agent interaction by injecting a task prompt into the GroupChatManager.

It effectively defines the primary objective or problem statement around which all subsequent agent behaviors—planning, reasoning, collaboration, and tool use—will evolve.


In [ ]:
task_prompt = "Please propose a new AI product idea"

user.initiate_chat(
    manager,
    message=task_prompt,
)

## 💡 4. Put Everything Together - Building a Multi-Agent AI Startup

We will simulate a collaborative AI startup team composed of multiple specialized agents, each responsible for a distinct role in the product development lifecycle. These agents will work together to brainstorm ideas, define the vision, architect the solution, design the interface, implement the application, and provide critical feedback — just like a real-world startup.

👥 Agents Overview

- Founder: Initiates the task and triggers the multi-agent workflow.

- Product Manager (PM): Defines the product vision, user needs, and overall feature roadmap.

- Tech Lead: Translates the product idea into a technical plan, outlining system components and technology stack.

- UI/UX Designer: Designs the user experience by describing the layout, wireframes, and interaction flow.

- Frontend Developer: Implements the UI using technologies like React and Tailwind CSS.

- Backend Developer: Develops server-side logic, APIs, and integration with AI services.

- Quality Assurance (QA): Performs quality assurance by reviewing the outputs of other agents—including code, design, and architecture—and provides constructive feedback to improve functionality, reliability, and usability.


### 4.1 Import libraries and setup


In [ ]:
import os
from typing import List, Dict, Any
import logging
from dotenv import load_dotenv
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager

# Load environment variables from .env file
load_dotenv()

# Setup logging to see execution messages
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### 4.2 Define configuration loading function


In [ ]:
def load_config() -> List[Dict[str, Any]]:
    """Load Azure OpenAI configuration from environment variables."""
    # List all required environment variables
    required_vars = [
        "AZURE_OPENAI_MODEL",  # The model name (e.g., gpt-4)
        "AZURE_OPENAI_API_KEY",  # Your Azure OpenAI API key
        "AZURE_OPENAI_ENDPOINT",  # Your Azure OpenAI endpoint URL
        "AZURE_OPENAI_API_VERSION",  # API version to use
    ]

    # Check if any variables are missing
    missing = [v for v in required_vars if not os.getenv(v)]
    if missing:
        raise ValueError(
            f"Missing required environment variables: {', '.join(missing)}"
        )

    # Return configuration for Azure OpenAI
    return [
        {
            "model": os.getenv("AZURE_OPENAI_MODEL"),
            "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
            "base_url": os.getenv("AZURE_OPENAI_ENDPOINT"),
            "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
            "api_type": "azure",
        }
    ]

### 4.3 Define function to create all agents


In [ ]:
def create_agents(config_list: List[Dict[str, Any]]) -> List[Any]:
    """Create all agents for the AI startup team."""
    # Create the Founder agent (User Proxy)
    founder = UserProxyAgent(
        name="Founder",
        human_input_mode="TERMINATE",  # Allows termination with TERMINATE
        max_consecutive_auto_reply=10,  # Maximum auto-replies before human input
        is_termination_msg=lambda msg: msg.get("content", "")
        .strip()
        .endswith("TERMINATE"),  # Function to detect termination messages
        code_execution_config={
            "work_dir": "output",  # Directory for code execution outputs
            "use_docker": False,  # Don't use Docker for code execution
        },
        system_message="You are the Founder of the AI startup. You initiate tasks and trigger the multi-agent workflow. Reply TERMINATE when done to end the conversation.",
    )

    # Create Product Manager agent
    product_manager = AssistantAgent(
        name="ProductManager",
        llm_config={
            "config_list": config_list,
            "temperature": 0.3,  # Slightly creative but mostly focused
            "max_tokens": 512,  # Maximum response length
        },
        system_message="""You are the Product Manager. Your role is to define the product vision, user needs, and overall feature roadmap.
        You translate customer problems into product requirements. Focus on:
        1. Target user identification
        2. User requirements and pain points
        3. Feature prioritization
        4. Product success metrics
        5. Market positioning
        Be specific and provide actionable insights that the technical team can implement.""",
    )

    # Create Tech Lead agent
    tech_lead = AssistantAgent(
        name="TechLead",
        llm_config={
            "config_list": config_list,
            "temperature": 0.2,  # More deterministic for technical decisions
            "max_tokens": 512,
        },
        system_message="""You are the Tech Lead. Your role is to translate the product idea into a technical plan.
        You outline system components and technology stack. Focus on:
        1. System architecture
        2. Technology selection
        3. Technical constraints and solutions
        4. Implementation strategy
        5. Scalability and performance considerations
        Provide specific technical recommendations that balance innovation with practicality.""",
    )

    # Create UI/UX Designer agent
    ui_ux_designer = AssistantAgent(
        name="UI_UX_Designer",
        llm_config={
            "config_list": config_list,
            "temperature": 0.3,  # Slightly creative for design ideas
            "max_tokens": 512,
        },
        system_message="""You are the UI/UX Designer. Your role is to design the user experience.
        You describe layouts, wireframes, and interaction flow. Focus on:
        1. User journey mapping
        2. Interface layout suggestions
        3. Visual design principles
        4. Interaction patterns
        5. Accessibility considerations
        Provide concrete design suggestions that create intuitive and engaging user experiences.""",
    )

    # Create Frontend Developer agent
    frontend_developer = AssistantAgent(
        name="FrontendDeveloper",
        llm_config={
            "config_list": config_list,
            "temperature": 0.2,  # More deterministic for coding
            "max_tokens": 512,
        },
        system_message="""You are the Frontend Developer. Your role is to implement the UI.
        You specialize in technologies like React and Tailwind CSS. Focus on:
        1. Component architecture
        2. State management
        3. Responsive design implementation
        4. Frontend performance optimization
        5. Frontend testing strategies
        Provide specific implementation approaches for building robust frontend solutions.""",
    )

    # Create Backend Developer agent
    backend_developer = AssistantAgent(
        name="BackendDeveloper",
        llm_config={
            "config_list": config_list,
            "temperature": 0.2,  # More deterministic for backend logic
            "max_tokens": 512,
        },
        system_message="""You are the Backend Developer. Your role is to develop server-side logic, APIs, and integration with AI services.
        Focus on:
        1. API design and implementation
        2. Database schema
        3. Server architecture
        4. Authentication and authorization
        5. AI service integration
        Provide specific implementation strategies for robust and scalable backend systems.""",
    )

    # Create Quality Assurance agent
    qa_engineer = AssistantAgent(
        name="QA_Engineer",
        llm_config={
            "config_list": config_list,
            "temperature": 0.2,  # More deterministic for finding issues
            "max_tokens": 512,
        },
        system_message="""You are the Quality Assurance Engineer. Your role is to review outputs of other agents and provide constructive feedback.
        Focus on:
        1. Identifying potential issues
        2. Testing strategies
        3. Edge cases
        4. User experience evaluation
        5. Performance considerations
        Provide specific feedback to improve functionality, reliability, and usability.""",
    )

    # Return all agents in the desired speaking order
    return [
        founder,
        product_manager,
        tech_lead,
        ui_ux_designer,
        frontend_developer,
        backend_developer,
        qa_engineer,
    ]

### 4.4 Define function to run the startup team


In [ ]:
def run_startup_team(task: str, max_round: int = 10):
    """Run the AI startup team with the given task."""
    try:
        # Load Azure OpenAI configuration
        logger.info("Loading configuration...")
        config_list = load_config()

        # Create all agents for the team
        logger.info("Creating AI startup team...")
        agents = create_agents(config_list)

        # Create group chat to facilitate conversation between agents
        groupchat = GroupChat(
            agents=agents,
            messages=[],
            max_round=max_round,  # Maximum conversation rounds
            speaker_selection_method="auto",  # AutoGen decides which agent speaks next
        )

        # Create manager to coordinate the group chat
        manager = GroupChatManager(
            groupchat=groupchat, llm_config={"config_list": config_list}
        )

        # Start the conversation with the specified task
        logger.info("Starting AI startup team conversation...")
        agents[0].initiate_chat(
            manager,
            message=task,
        )

        logger.info("AI startup team conversation completed.")

    except Exception as e:
        logger.error(f"Error running AI startup team: {e}")
        raise

### 4.5 Define example task and start the conversation (Demo only)


In [ ]:
# Define an AI product task for the team to work on
task = """
We want to build an AI-powered productivity tool that:
1. Automatically organizes and summarizes emails, messages, and documents
2. Generates actionable to-do lists from communications
3. Prioritizes tasks based on urgency and importance
4. Integrates with common productivity tools (Gmail, Slack, MS Office)
5. Provides a simple and intuitive interface for users

Let's collaborate to define, design, and plan implementation for this product.

For this demo, I'd like each team member to share ONLY their high-level thoughts based on their expertise.
"""

# Run the AI startup team with this task
run_startup_team(task, max_round=15)

## 🗂️ 5. More Examples and Resources


### 📚 More Examples

- [Data Analysis Agents](Examples/data_analyst_agent.py)
- [Tutorial Lab Agent](Examples/tutorial_lab_agent.py)
- [Architecture Design Agents](Examples/architecture_design_agent/README.md)


### 🔗 Resources

- [AutoGen GitHub](https://github.com/microsoft/autogen)
- [Official Docs](https://microsoft.github.io/autogen/stable/)
